In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, cohen_kappa_score, mean_squared_error, r2_score, roc_auc_score
from sklearn.metrics import roc_curve
import time

In [15]:
# Configuração de seed para reprodutibilidade
seed = 42
np.random.seed(seed)

In [16]:
# Carregar os dados
df = pd.read_csv('../data/weaving_rejection_dataset_updated.csv')

In [17]:
# Separar as features (X) e a variável alvo (y)
X = df.drop('Rejection', axis=1)
y = df['Rejection']

In [18]:
# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [19]:
# Escalar os dados (normalização)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Função para avaliar o modelo
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    # Binarização das previsões (transformando para classificação)
    y_pred_class = (y_pred >= 0.5).astype(int)  # Limiar para binarizar
    
    # Métricas de classificação
    accuracy = accuracy_score(y_test, y_pred_class)
    kappa = cohen_kappa_score(y_test, y_pred_class)
    
    # Calcular o AUC para modelo binário
    if len(np.unique(y_test)) == 2:
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred)
    else:
        auc = None  # Não aplicável para multiclass
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")
    if auc is not None:
        print(f"AUC: {auc:.4f}")
    else:
        print("AUC não disponível para multiclasses.")
    
    return accuracy, kappa, auc, r2

In [21]:
# Modelo sem PCA
print("Treinando modelo SVR sem PCA...")
svr_model = SVR(kernel='rbf')
start_time = time.time()
svr_model.fit(X_train_scaled, y_train)
elapsed_time = time.time() - start_time

Treinando modelo SVR sem PCA...


In [22]:
print(f"Tempo de treinamento: {time.strftime('%H:%M:%S', time.gmtime(elapsed_time))}")
print("Avaliação do modelo SVR sem PCA:")
evaluate_model(svr_model, X_test_scaled, y_test)

Tempo de treinamento: 00:00:14
Avaliação do modelo SVR sem PCA:
Mean Squared Error: 31380.1781
R² Score: 0.1108
Accuracy: 0.0756
Cohen's Kappa: 0.0502
AUC não disponível para multiclasses.


(0.07564743298500681, 0.050164126424834365, None, 0.1107530106048551)

In [23]:
# Aplicar PCA com 95% de variância explicada
print("\nAplicando PCA com 95% de variância explicada...")
pca = PCA(0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


Aplicando PCA com 95% de variância explicada...


In [24]:
# Modelo com PCA
print("Treinando modelo SVR com PCA...")
svr_model_pca = SVR(kernel='rbf')
start_time_pca = time.time()
svr_model_pca.fit(X_train_pca, y_train)
elapsed_time_pca = time.time() - start_time_pca

Treinando modelo SVR com PCA...


In [25]:
print(f"Tempo de treinamento com PCA: {time.strftime('%H:%M:%S', time.gmtime(elapsed_time_pca))}")
print("Avaliação do modelo SVR com PCA:")
evaluate_model(svr_model_pca, X_test_pca, y_test)

Tempo de treinamento com PCA: 00:00:12
Avaliação do modelo SVR com PCA:
Mean Squared Error: 31234.8003
R² Score: 0.1149
Accuracy: 0.0756
Cohen's Kappa: 0.0500
AUC não disponível para multiclasses.


(0.07564743298500681, 0.0499566130606357, None, 0.11487270514998904)